In [9]:
import pandas as pd
import numpy as np

dftrain = pd.read_csv("LabelEncoded.csv")
dftrain
cat_cols = ['Street','City','County','State','Airport_Code',
            'Wind_Direction','Weather_Condition','Sunrise_Sunset']

for col in cat_cols:
    dftrain[col] = dftrain[col].astype('category')

df1 = dftrain.drop(columns=['Unnamed: 0'])
X = df1.drop(columns=['Severity'])
y = df1['Severity']
y = y - 1   # shift labels if needed


In [12]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)
X_train

,Year,Start_Lat,Start_Lng,Distance(mi),Street,City,County,State,Airport_Code,Temperature(F),Wind_Chill(F),Visibility(mi),Wind_Direction,Weather_Condition,Traffic_Signal,Sunrise_Sunset,TimeDiff
185766,1,33.506004,-84.230499,0.00,18921,5864,500,9,135,45.0,41.8,10.0,15,2,0,1,29.500000
204523,1,43.206711,-77.675842,0.00,40302,5178,755,32,1304,39.0,31.0,10.0,15,2,0,1,29.750000
21597,0,39.598373,-84.234108,0.00,19717,3811,759,33,979,39.9,33.9,7.0,22,46,0,1,29.650000
2758,0,34.392521,-118.472832,0.01,16596,4232,656,3,23,53.0,53.0,10.0,15,47,0,1,60.000000
146354,2,39.772060,-86.102165,0.00,25793,2886,689,13,516,37.0,28.9,5.0,8,46,1,0,29.800000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119879,2,36.402393,-87.024963,0.00,19148,4836,959,40,308,42.1,37.1,10.0,2,2,0,1,28.700000
103694,2,39.699554,-75.556068,0.00,17238,4153,784,7,773,44.1,41.5,10.0,3,46,0,1,29.483333
131932,2,43.048462,-83.557320,0.00,25166,1465,418,20,562,43.0,40.3,10.0,14,2,0,1,29.866667
146867,2,41.060398,-74.122826,0.00,12916,5017,91,29,278,32.0,27.5,0.5,4,59,0,1,29.033333


In [7]:
from xgboost import XGBClassifier

model = XGBClassifier(
    n_estimators=300,
    max_depth=3,
    eta=0.2,
    colsample_bytree=0.67,
    n_jobs=-1,
    enable_categorical=True,
    verbosity=0
)

model.fit(X_train, y_train)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.67, device=None, early_stopping_rounds=None,
              enable_categorical=True, eta=0.2, eval_metric=None,
              feature_types=None, feature_weights=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=300, n_jobs=-1, ...)

In [13]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

y_pred = model.predict(X_test)

acc = accuracy_score(y_test, y_pred)
print("Test Accuracy:", acc * 100, "%")

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

print("Classification Report:")
print(classification_report(y_test, y_pred))


Test Accuracy: 87.06279318574603 %
Confusion Matrix:
[[    0    15     1     0]
 [    2 28136  2570   110]
 [    0  2994 13788    60]
 [    0   406   130   392]]
Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        16
           1       0.89      0.91      0.90     30818
           2       0.84      0.82      0.83     16842
           3       0.70      0.42      0.53       928

    accuracy                           0.87     48604
   macro avg       0.61      0.54      0.56     48604
weighted avg       0.87      0.87      0.87     48604



In [14]:
model.get_booster().save_model("xgboost_model.json")